In [4]:
%run SU3_implementations.ipynb

In [3]:
# Function to compute the couplings of spins J1 and J2
def CoupleSpin(J1, J2):
    return [i for i in np.arange(abs(J1-J2), abs(J1+J2)+1)]

In [4]:
# Obtain the irreps of SO(3) contained in SU(3)
def DecomposeSU3SO3(lam, mu):
    
    # Array to store allowed K and L
    SU3SO3 = []
    
    # Values of K
    K = [i for i in range(min(lam, mu),-1,-2)]
    
    for k in K:
        
        # Cases depending on the value of K
        if k == 0:
            L = [i for i in range(max(lam, mu),-1,-2)]
        else:
            L = [i for i in range(k, k+max(lam,mu)+1, 1)]
            
        SU3SO3.append([k, L])
            
    return SU3SO3

In [5]:
# Obtain the irreps of U(1) contained in SU(2)
def DecomposeSU2U1(J): 
    return [M for M in np.arange(-J, J+1,1)]

In [6]:
# Function to obtain all states of given irreps of U(N)
def Completeness(fpm,fpp,fnm,fnp):
    
    # U2 conjugate values
    U2pm = Conjugate(fpm)
    U2pp = Conjugate(fpp)
    U2nm = Conjugate(fnm)
    U2np = Conjugate(fnp)
    
    # Spin of each shell
    SU2pm = (U2pm[0] - U2pm[1])/2
    SU2pp = (U2pp[0] - U2pp[1])/2 
    SU2nm = (U2nm[0] - U2nm[1])/2     
    SU2np = (U2np[0] - U2np[1])/2 
    
    # Allowed spin of each particle space
    SU2p = CoupleSpin(SU2pm, SU2pp)
    SU2n = CoupleSpin(SU2nm, SU2np)
    
    # SU3 irreps contained in the U(N) irreps
    SU3pm = UNtoU3(5, fpm)
    SU3pp = UNtoU3(6, fpp)
    SU3nm = UNtoU3(6, fnm)
    SU3np = UNtoU3(7, fnp)

    # Complete modelspace array
    ModelSpaceArray = []

    # Coupled irreps of SU3 per nucleon
    for index_pm, row_pm in SU3pm.iterrows(): 
        for index_pp, row_pp in SU3pp.iterrows(): 
            
            # Protons
            SU3p = SU3prod(row_pm["lam"], row_pm["mu"], row_pp["lam"], row_pp["mu"])
            
            for index_nm, row_nm in SU3nm.iterrows(): 
                for index_np, row_np in SU3np.iterrows(): 
                    
                     # Neutrons
                    SU3n = SU3prod(row_nm["lam"], row_nm["mu"], row_np["lam"], row_np["mu"])

                    for index_p, row_p in SU3p.iterrows():
                        for index_n, row_n in SU3n.iterrows():  
                
                            # Full space
                            SU3 = SU3prod(row_p["lam"], row_p["mu"], row_n["lam"], row_n["mu"])
                    
                            for index, row in SU3.iterrows(): 
                                
                                # Angular momentum and K labels
                                SU3SO3 = DecomposeSU3SO3(row["lam"], row["mu"])
                                
                                for sp in SU2p:
                                    for sn in SU2n:
                                        
                                        # Spin labels
                                        SU2 = CoupleSpin(sp, sn)
                                        
                                        for su3so3 in SU3SO3:
                                            for s in SU2:
                                                for l in su3so3[1]:
                                                
                                                    # Total angular momentum
                                                    SU2J = CoupleSpin(s, l)
                                                    
                                                    for j in SU2J:
                                                        
                                                        # Addition of state to the array
                                                        # απ-   (λ,μ)π-   απ+   (λ,μ)π+   αν-   (λ,μ)ν-   αν+   (λ,μ)ν+   ρπ   (λ,μ)π   ρν   (λ,μ)ν   ρ   (λ,μ)   K   L   Sπ   Sν   S   J                                        
                                                        ModelSpaceArray.append([row_pm["num"], row_pm["lam"], row_pm["mu"], row_pp["num"], row_pp["lam"], row_pp["mu"], row_nm["num"], row_nm["lam"], row_nm["mu"], row_np["num"], row_np["lam"], row_np["mu"], row_p["mult"], row_p["lam"], row_p["mu"], row_n["mult"], row_n["lam"], row_n["mu"], row["mult"], row["lam"], row["mu"], su3so3[0], l, sp, sn, s, j])

                                                        # Projections
                                                        #U1M = DecomposeSU2U1(j)
                                                        
                                                        #for m in U1M:
                                                           
                                                            # Addition of state to the array
                                                            # απ-   (λ,μ)π-   απ+   (λ,μ)π+   αν-   (λ,μ)ν-   αν+   (λ,μ)ν+   ρπ   (λ,μ)π   ρν   (λ,μ)ν   ρ   (λ,μ)   K   L   Sπ   Sν   S   J   M                                      
                                                            #ModelSpaceArray.append([ row_pm["lam"], row_pm["mu"], row_pp["lam"], row_pp["mu"], row_nm["lam"], row_nm["mu"], row_np["lam"], row_np["mu"], row_p["mult"], row_p["lam"], row_p["mu"], row_n["mult"], row_n["lam"], row_n["mu"], row["mult"], row["lam"], row["mu"], su3so3[0], l, sp, sn, s, j, m])
    
    values_df = pd.DataFrame(np.array(ModelSpaceArray), columns = ["αpm", "λ_pm", "μ_pm", "αpp","λ_pp", "μ_pp", "αnm","λ_nm", "μ_nm", "αnp","λ_np", "μ_np", "ρp", "λ_p", "μ_p", "ρn", "λ_n", "μ_n", "ρ", "λ", "μ", "K", "L", "Sp", "Sn", "S", "J"])
 
    return values_df

In [7]:
Completeness([2],[0],[2],[0])

,αpm,λ_pm,μ_pm,αpp,λ_pp,μ_pp,αnm,λ_nm,μ_nm,αnp,...,μ_n,ρ,λ,μ,K,L,Sp,Sn,S,J
0,1.0,10.0,0.0,1.0,0.0,0.0,1.0,12.0,0.0,1.0,...,0.0,1.0,22.0,0.0,0.0,22.0,0.0,0.0,0.0,22.0
1,1.0,10.0,0.0,1.0,0.0,0.0,1.0,12.0,0.0,1.0,...,0.0,1.0,22.0,0.0,0.0,20.0,0.0,0.0,0.0,20.0
2,1.0,10.0,0.0,1.0,0.0,0.0,1.0,12.0,0.0,1.0,...,0.0,1.0,22.0,0.0,0.0,18.0,0.0,0.0,0.0,18.0
3,1.0,10.0,0.0,1.0,0.0,0.0,1.0,12.0,0.0,1.0,...,0.0,1.0,22.0,0.0,0.0,16.0,0.0,0.0,0.0,16.0
4,1.0,10.0,0.0,1.0,0.0,0.0,1.0,12.0,0.0,1.0,...,0.0,1.0,22.0,0.0,0.0,14.0,0.0,0.0,0.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5008,1.0,2.0,4.0,1.0,0.0,0.0,1.0,0.0,6.0,1.0,...,6.0,1.0,3.0,2.0,0.0,3.0,0.0,0.0,0.0,3.0
5009,1.0,2.0,4.0,1.0,0.0,0.0,1.0,0.0,6.0,1.0,...,6.0,1.0,3.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0
5010,1.0,2.0,4.0,1.0,0.0,0.0,1.0,0.0,6.0,1.0,...,6.0,1.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0
5011,1.0,2.0,4.0,1.0,0.0,0.0,1.0,0.0,6.0,1.0,...,6.0,1.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0
